# Social Interaction Ethogram Built

Create a panda structure with all relevant parameters for ethogram construction. 

Parameters are based definitions of in de Chaumon et al. Nature 2012. 'Computerized video analysis of social interactions in mice'.

''' May 25th 2021'''


In [1]:
import os
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from pathlib import Path
import csv
from collections import namedtuple
from sympy import *


In [2]:
def distance(point1,point2):
    xdiff = point1[0] - point2[0]
    ydiff = point1[1] - point2[1]
    dist = np.sqrt(xdiff*xdiff + ydiff*ydiff)
    return dist

def direction(point1,point2):
    x_diff = point1[0] - point2[0]
    y_diff = point1[1] - point2[1]
    direction = np.array([x_diff , y_diff])
    return direction

def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::
            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [3]:
def get_body_parameters(tracking = None, body_parts = None):
    
    '''
    This function will receive the final positions of body parts (after stimation of nose)
    It will have N*2 entraces, where N is the number of body parts per animal
    For starting point we will use the nose of the tracker and then refine it to another nose position
    using other information (as drive position, maybenose, accelerometer, etc).
    '''
    
    N = int(np.min(tracking.shape)/6)
    tpoints = tracking.shape[0]
    
    #separate animals tracking
    tracking1 = np.array([tracking[:,[i*3,i*3+1]] for i in range(N)])
    tracking2 = np.array([tracking[:,[i*3+N*3,i*3+1+N*3]] for i in range(N)])
    
    print(tracking1.shape)
    
    #compute center of mass taking into account all bodyparts
    cm1 =  np.mean(tracking1,axis=0)
    cm2 =  np.mean(tracking2,axis=0)
    
    #compute speed
    v1 = np.array([np.diff(cm1[:,i], prepend = 0) for i in range(2)]).T
    speed1 = np.linalg.norm(v1,axis = 0)
    v2 = np.array([np.diff(cm2[:,i], prepend = 0) for i in range(2)]).T
    speed2 = np.linalg.norm(v2,axis = 0)
    
    #compute head direction
    hd1= np.array([direction(tracking1[3,i,:],tracking1[0,i,:]) for i in range(tpoints)])
    hd2= np.array([direction(tracking2[3,i,:],tracking1[0,i,:]) for i in range(tpoints)])

    #compute body direction
    bd1= np.array([direction(tracking1[4,i,:],tracking1[3,i,:]) for i in range(tpoints)])
    bd2= np.array([direction(tracking2[4,i,:],tracking1[3,i,:]) for i in range(tpoints)])

    parameters = {'cm1' : cm1 , 'cm2': cm2, 'head_direction1': hd1 , 'head_direction2': hd2 , 
                  'speed1' : speed1, 'speed2': speed2 ,
                  'body_direction1': bd1, 'body_direction2': bd2}

    return parameters

In [13]:
from sklearn.linear_model import LinearRegression

def circle_center(point1,point2,point3):
    
    '''
    Compute the center of the circle along the axis that is perpendicular
    to the ears and intersects nose positions
    input : point1 -> Nose position
            point2 -> RightEar position
            point3 -> LeftEar position
    return : C1 center of circle position
    '''
    
    point4 = (point3 + point2)/2
    C1 = (point1 + point4)/2
    
    return C1

def ellipse_focus(point1,point2,point3):
    '''
    Compute the focus of the elipse along the the axis that optimal fits
    neck, com and tail positions.
    input : point1 -> Neck position
            point2 -> Center of mass position
            point3 -> tail position
    return : F1 and F2 focus of the elipsoide
    
    '''
    X = np.array([point1[:,0],point2[:,0],point3[:,0]])
    y = np.array([point1[:,1],point2[:,1],point3[:,1]])
    reg = LinearRegression().fit(X, y)
    
    F1_x = (point1[:,0] + point2[:,0])/2
    F1_y = reg.predict(np.array([F1_x]))
    
    F1 = np.array([F1_x,F1_y])
    
    F2_x = (point2[:,0] + point3[:,0])/2
    F2_y = reg.predict(np.array([F2_x]))
    
    F2 = np.array([F2_x,F2_y])
    return F1,F2


def ellipse_model(points_set = None , axis_length = [0,0]):
    
    '''
    Compute center and axis lenght of the elipse along the the axis that optimal fits
    neck, com and tail positions.
    input : set of points with at least the next points information
            point1 -> Neck position
            point2 -> Center of mass position
            point3 -> tail position
    return : C center of elipsoide, angle of principal axis and axis1 and axis2 length of the elipsoide
    
    '''
    #compute line for principal axis of the elipsoide
    X = points_set[:,0]
    y = points_set[:,1]
    reg = LinearRegression().fit(X.reshape(-1,1), y.reshape(-1,1))

    #compute center of the elipsoude
    C_x = np.mean(points_set[:,0])
    C_y = float(reg.predict(C_x.reshape(-1,1)))
    center = np.array([C_x,C_y])
    
    # compute angle of principal axis
    bias = reg.predict(np.array([0]).reshape(-1,1))
    angle =  float(np.arctan2(center[1] - bias , center[0]))
    
    #compute axis1 and axis2 length 
    axis1 = np.linalg.norm(points_set[-1,:]-points_set[0,:])/2  #big axis is variable
    axis2 = axis_length[1] #small axis is fixed
    
    #compute ellipse focus based on axis length and angle
    gamma = np.sqrt(abs(axis1**2 - axis2**2))
    focus1_x = C_x + gamma * np.cos(angle)
    focus1_y = C_y + gamma * np.sin(angle)

    focus2_x = C_x - gamma * np.cos(angle)
    focus2_y = C_y - gamma * np.sin(angle)
    
    elipse = namedtuple('elipse', ['reg', 'center', 'angle', 'axis1', 'axis2'])
    return elipse(reg, center , angle, axis1, axis2)

def mouse_model(tracking = None):
    
    '''
    Create a model of a mouse based on ellipses that are constructud by taking into account body positions.
    '''
    
    head_parts = tracking[[0,3],:,:]
    head_model = [ellipse_model(points_set = head_parts[:,i,:],axis_length = [0,10]) for i in range(head_parts.shape[1])]
    
    #point1 = tracking[3,:,:]
    #point2 = tracking[4,:,:]
    #point3 = tracking[5,:,:]
    #body = ellipse_focus(point1,point2,point3)
    
    body_parts = tracking[[3,4,5],:,:]
    body_model = [ellipse_model(points_set = body_parts[:,i,:], axis_length = [0,20]) for i in range(body_parts.shape[1])]
    
    model = namedtuple('mouse_model', ['head', 'body'])
    return model(head_model , body_model)

In [14]:
tracking_path = Path('/home/melisa/Documents/social_interaction/')
tracking_path.exists()
input_file_name = 'raw120sDLC_resnet50_social_interaction_labelmei19shuffle1_694000.csv'
input_file_path = tracking_path / input_file_name
tracking_DFrame = pd.read_csv(input_file_path)

In [15]:
#tracking_DFrame

In [16]:
body_parts = [tracking_DFrame.iloc[0][i*3+1] for i in range(int(len(tracking_DFrame.iloc[0])/3))]
body_part_structure = [tracking_DFrame.iloc[1][i+1] for i in range(3)]

In [17]:
tracking_DFrame = tracking_DFrame.iloc[2:]
tracking_DFrame = tracking_DFrame.astype(float)
tracking_data = tracking_DFrame.to_numpy()

In [18]:
body_tracking = tracking_data[:,1+5*3:];
parameters = get_body_parameters(tracking = body_tracking);
#df = pd.DataFrame(list(parameters.items()),columns = ['cm1','cm2','head_direction1','head_direction2','speed1','speed2','body_direction1','body_directiond2']) 


(6, 3600, 2)


In [19]:
tracking = body_tracking
N = int(np.min(tracking.shape)/6)
#separate animals tracking
tracking1 = np.array([tracking[:,[i*3,i*3+1]] for i in range(N)])
tracking2 = np.array([tracking[:,[i*3+N*3,i*3+1+N*3]] for i in range(N)])
    

In [20]:
model1 = mouse_model(tracking = tracking1)
model2 = mouse_model(tracking = tracking2)

In [27]:
model1.head[0].center[0]

211.48743438720703

In [ ]:
## Solve the intersection problem using elipse formulas, lagrange multipliers and symbolic python

x1, x2, y1, y2, l1, l2 = symbols('x1 x2 y1 y2 l1 l2')
frame_n = 100

a1 = model1.body[frame_n].axis2**2
b1 = model1.body[frame_n].axis1**2
c1 = 0
d1 = -2*model1.body[0].axis2**2*model1.body[frame_n].center[0]
e1 = -2*model1.body[0].axis2**2*model1.body[frame_n].center[1]
f1 = a1* model1.body[frame_n].center[0]**2 + b1 * model1.body[frame_n].center[1]**2 - a1*b1


a2 = model2.body[frame_n].axis2**2
b2 = model2.body[frame_n].axis1**2
c2 = 0
d2 = -2*model2.body[0].axis2**2*model2.body[frame_n].center[0]
e2 = -2*model2.body[0].axis2**2*model2.body[frame_n].center[1]
f2 = a1* model2.body[frame_n].center[0]**2 + b1 * model2.body[frame_n].center[1]**2 - a1*b1

solve([Eq(2*(x1-x2) + l1 *(2*a1*x1 + c1*y1 + d1),0),
   Eq(2*(y1-y2) + l1 *(2*b1*y1 + d1*x1 + e1), 0),
   Eq(2*(x2-x1) + l2 *(2*a2*x2 + c2*y2 + d2),0),
   Eq(2*(y2-y2) + l2 *(2*b2*y2 + d2*x2 + e2), 0),
   Eq(a1*x1**2+b1*y1**2+d1*x1+e1*y1+f1,0),
   Eq(a2*x2**2+b2*y2**2+d2*x2+e2*y2+f2,0)], [x1,x2,y1,y2,l1,l2], simplify=False)

In [381]:
import cv2
input_video_name = tracking_path / 'raw120s.mp4'
output_video = tracking_path / 'ellipse.avi'

cap = cv2.VideoCapture(str(input_video_name))
#cap.set(cv2.CAP_PROP_POS_FRAMES, frame_n)
output = cv2.VideoWriter(str(output_video),cv2.VideoWriter_fourcc(*'MJPG'),30,(700,700))

for frame_n in range(3600):
    
    r , frame = cap.read()    
    frame = np.zeros((700,700,3),dtype = 'uint8')
    p1 = (0, model1.body[frame_n].reg.intercept_)
    p2 = (700, 700* model1.body[frame_n].reg.coef_ + model1.body[frame_n].reg.intercept_)
    cv2.line(frame, p1, p2 , color = (255,255,255));
    p1 = (0, model1.head[frame_n].reg.intercept_)
    p2 = (700, 700* model1.head[frame_n].reg.coef_ + model1.head[frame_n].reg.intercept_)
    cv2.line(frame, p1, p2 , color = (255,255,255));
    
    p1 = (0, model2.body[frame_n].reg.intercept_)
    p2 = (700, 700* model2.body[frame_n].reg.coef_ + model2.body[frame_n].reg.intercept_)
    cv2.line(frame, p1, p2 , color = (0,0,255));
    p1 = (0, model2.head[frame_n].reg.intercept_)
    p2 = (700, 700* model2.head[frame_n].reg.coef_ + model2.head[frame_n].reg.intercept_)
    cv2.line(frame, p1, p2 , color = (0,0,255));

    
    center1 = model1.body[frame_n].center.astype(int)
    center2 = model2.body[frame_n].center.astype(int)
    
    #print(tuple(center1))
    #print((model1.body[frame_n].axis1.astype(int),model1.body[frame_n].axis2))

    cv2.circle(frame, tuple(center1), 10, color = (255,0,0), thickness = 1)
    cv2.ellipse(frame, tuple(center1),(model1.body[frame_n].axis1.astype(int),model1.body[frame_n].axis2) , math.degrees(model1.body[frame_n].angle)  , 0, 360, color = (0,0,255), thickness = 5);
    
    cv2.circle(frame, tuple(center2), 10, color = (255,0,0), thickness = 1)
    cv2.ellipse(frame, tuple(center2),(model2.body[frame_n].axis1.astype(int),model2.body[frame_n].axis2) , math.degrees(model2.body[frame_n].angle)  , 0, 360, color = (0,255,0), thickness = 5);
    
    
    center1 = model1.head[frame_n].center.astype(int)
    center2 = model2.head[frame_n].center.astype(int)
    
    #print(tuple(center1))
    #print((model1.body[frame_n].axis1.astype(int),model1.body[frame_n].axis2))

    cv2.circle(frame, tuple(center1), 10, color = (255,0,0), thickness = 1)
    cv2.ellipse(frame, tuple(center1),(model1.head[frame_n].axis1.astype(int),model1.head[frame_n].axis2) , math.degrees(model1.head[frame_n].angle)  , 0, 360, color = (0,0,255), thickness = 5);
    
    cv2.circle(frame, tuple(center2), 10, color = (255,0,0), thickness = 1)
    cv2.ellipse(frame, tuple(center2),(model2.head[frame_n].axis1.astype(int),model2.head[frame_n].axis2) , math.degrees(model2.head[frame_n].angle)  , 0, 360, color = (0,255,0), thickness = 5);
    
    output.write(frame)
    